In [1]:
!pip install biopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 1.0 MB/s eta 0:00:00


Based on https://www.kaggle.com/code/hengck23/lb0-335-deepdgg-server-benchmark and https://www.kaggle.com/code/lucasmorin/nesp-changes-eda-and-baseline

In [2]:
import Levenshtein
import pandas as pd
from biopandas.pdb import PandasPdb
import numpy as np
from Bio.SubsMat import MatrixInfo
from scipy import stats

/opt/conda/lib/python3.7/site-packages/Bio/SubsMat/__init__.py:131: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  BiopythonDeprecationWarning,


In [3]:
cap_sub_score_zero = True
ddg_filna_score = -0.25
submit_col = 'rank_pow'
sigmoid_norm_factor = 3

In [4]:
class paths:
    TRAIN = "/kaggle/input/novozymes-enzyme-stability-prediction/train.csv"
    TEST = "/kaggle/input/novozymes-enzyme-stability-prediction/test.csv"
    SUBMISSION = "/kaggle/input/novozymes-enzyme-stability-prediction/sample_submission.csv"
    PDB_FILE = "/kaggle/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"

In [5]:
base = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

In [6]:
# source: https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/354783
def get_mutation_info(_row, _wildtype=base):
    terminology_map = {"replace":"substitution", "insert":"insertion", "delete":"deletion"}
    req_edits = Levenshtein.editops(_wildtype, _row["protein_sequence"])
    _row["n_edits"] = len(req_edits)

    if _row["n_edits"]==0:
        _row["edit_type"] = _row["edit_idx"] = _row["old_aa"] = _row["new_aa"] = pd.NA
    else:
        _row["edit_type"] = terminology_map[req_edits[0][0]]
        _row["edit_idx"] = req_edits[0][1]
        _row["old_aa"] = _wildtype[_row["edit_idx"]]
        _row["new_aa"] = _row["protein_sequence"][req_edits[0][2]] if _row["edit_type"]!="deletion" else pd.NA
    return _row

def revert_to_wildtype(protein_sequence, edit_type, edit_idx, old_aa, new_aa):
    if pd.isna(edit_type):
        return protein_sequence
    elif edit_type!="insertion":
        new_wildtype_base = protein_sequence[:edit_idx]
        if edit_type=="deletion":
            new_wildtype=new_wildtype_base+old_aa+protein_sequence[edit_idx:]
        else:
            new_wildtype=new_wildtype_base+old_aa+protein_sequence[edit_idx+1:]
    else:
        new_wildtype=protein_sequence[:edit_idx]+old_aa+protein_sequence[edit_idx:]
    return new_wildtype




#helper function
def read_list_from_file(list_file):
    with open(list_file) as f:
        lines  = f.readlines()
    return lines


In [7]:
test_df = pd.read_csv(paths.TEST)
test_df = test_df.apply(get_mutation_info, axis=1)
test_df.loc[test_df.edit_type.isna(), 'edit_type'] = 'nothing'
test_df.head()

,seq_id,protein_sequence,pH,data_source,n_edits,edit_type,edit_idx,old_aa,new_aa
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,E
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,K
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,1,deletion,16,L,<NA>
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,C
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,F


In [8]:
pdb_df =  PandasPdb().read_pdb(paths.PDB_FILE)
pdb_df.df.keys()

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])

In [9]:
atom_df = pdb_df.df['ATOM']
atom_df['residue_number_0based'] = atom_df['residue_number'] - 1
map_number_to_b = atom_df.groupby('residue_number_0based').b_factor.mean()
test_df['b_factor'] = test_df.edit_idx.map(map_number_to_b).fillna(0)
test_df.loc[test_df['edit_type']=='deletion', 'new_aa'] = '-'
test_df.loc[test_df['edit_type']=='insertion', 'new_aa'] = '+'
test_df.loc[:,'mut_string'] =  test_df.old_aa+test_df.edit_idx.astype(str)+test_df.new_aa
test_df.head()

,seq_id,protein_sequence,pH,data_source,n_edits,edit_type,edit_idx,old_aa,new_aa,b_factor,mut_string
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,E,55.23,L16E
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,K,55.23,L16K
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,1,deletion,16,L,-,55.23,L16-
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,C,69.25,K17C
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,F,69.25,K17F


In [10]:
ddg = read_list_from_file('../input/submit-novozymes-00/wildtype_structure_prediction_af2.deepddg.ddg.txt')

header = ddg[0]
data = [s.split() for s in ddg[1:]]

df = pd.DataFrame(data, columns = ['chain', 'WT', 'ResID', 'Mut', 'ddG'])
df.ddG = df.ddG.astype(np.float32)
df.ResID = df.ResID.astype(int)  
df.loc[:,'location'] = df.ResID -1  #change to 0-indexing
df.loc[:,'mut_string'] = df.WT+df.location.astype(str)+df.Mut
df.head()

,chain,WT,ResID,Mut,ddG,location,mut_string
0,A,V,1,A,-0.119,0,V0A
1,A,V,1,R,-0.132,0,V0R
2,A,V,1,N,-0.155,0,V0N
3,A,V,1,D,-0.158,0,V0D
4,A,V,1,C,-0.162,0,V0C


In [11]:
test_df = test_df.merge(df[['ddG','mut_string']], on='mut_string', how='left')
test_df.loc[test_df['ddG'].isna(), 'ddG'] = ddg_filna_score
test_df.head()

,seq_id,protein_sequence,pH,data_source,n_edits,edit_type,edit_idx,old_aa,new_aa,b_factor,mut_string,ddG
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,E,55.23,L16E,-0.226
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,K,55.23,L16K,-0.169
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,1,deletion,16,L,-,55.23,L16-,-0.250
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,C,69.25,K17C,-1.277
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,F,69.25,K17F,-1.353


In [12]:
sub_scores = []
sub_mat = MatrixInfo.blosum100
for i in range(len(test_df)):
    mut_type = test_df.edit_type.values[i]
    if mut_type == 'substitution':
        try:
            sub_score = sub_mat[(test_df.old_aa.values[i], test_df.new_aa.values[i])]
        except KeyError:
            sub_score = sub_mat[(test_df.new_aa.values[i], test_df.old_aa.values[i])]
    elif mut_type == 'nothing':
        sub_score = 0
    else:
        sub_score = -10
    sub_scores.append(sub_score)

In [13]:
test_df['sub_score'] = sub_scores
if cap_sub_score_zero:
    test_df.loc[test_df['sub_score'] > 0, 'sub_score'] = 0
test_df['score_adj'] = [ 1 - (1 / (1+np.exp(-x/sigmoid_norm_factor))) for x in sub_scores]
test_df['b_factor_adj'] = test_df['b_factor'] * test_df['score_adj'] 
test_df.head(5)

,seq_id,protein_sequence,pH,data_source,n_edits,edit_type,edit_idx,old_aa,new_aa,b_factor,mut_string,ddG,sub_score,score_adj,b_factor_adj
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,E,55.23,L16E,-0.226,-5,0.841131,46.455659
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,K,55.23,L16K,-0.169,-4,0.791391,43.708551
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,1,deletion,16,L,-,55.23,L16-,-0.250,-10,0.965555,53.327592
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,C,69.25,K17C,-1.277,-5,0.841131,58.248314
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,F,69.25,K17F,-1.353,-4,0.791391,54.803859


In [14]:
test_df['ddG_rank'] = stats.rankdata(test_df['ddG'])
test_df['b_factor_rank'] = stats.rankdata(-test_df['b_factor'])
test_df['b_factor_adj_rank'] = stats.rankdata(-test_df['b_factor_adj'])
test_df['sub_score_rank'] = stats.rankdata(test_df['sub_score'])
test_df.head()

,seq_id,protein_sequence,pH,data_source,n_edits,edit_type,edit_idx,old_aa,new_aa,b_factor,mut_string,ddG,sub_score,score_adj,b_factor_adj,ddG_rank,b_factor_rank,b_factor_adj_rank,sub_score_rank
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,E,55.23,L16E,-0.226,-5,0.841131,46.455659,2115.5,2408.0,2203.0,313.0
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,K,55.23,L16K,-0.169,-4,0.791391,43.708551,2190.5,2408.0,2212.0,704.5
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,1,deletion,16,L,-,55.23,L16-,-0.250,-10,0.965555,53.327592,2056.5,2408.0,2080.0,39.0
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,C,69.25,K17C,-1.277,-5,0.841131,58.248314,1032.0,2386.5,1725.5,313.0
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,F,69.25,K17F,-1.353,-4,0.791391,54.803859,987.0,2386.5,2053.5,704.5


In [15]:
test_df['rank_pow'] = test_df.apply(lambda x: np.power(x['b_factor_rank'] * x['sub_score_rank'] * x['ddG_rank'], 1/3), axis=1)
# test_df['rank_pow'] = test_df.apply(lambda x: np.power(x['b_factor_adj_rank'] * x['sub_score_rank'] * x['ddG_rank'], 1/3), axis=1)
#test_df['rank_pow'] = test_df.apply(lambda x: np.power(x['b_factor_rank'] * x['b_factor_adj_rank'], 1/2), axis=1)
test_df.head()

,seq_id,protein_sequence,pH,data_source,n_edits,edit_type,edit_idx,old_aa,new_aa,b_factor,mut_string,ddG,sub_score,score_adj,b_factor_adj,ddG_rank,b_factor_rank,b_factor_adj_rank,sub_score_rank,rank_pow
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,E,55.23,L16E,-0.226,-5,0.841131,46.455659,2115.5,2408.0,2203.0,313.0,1168.255811
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,16,L,K,55.23,L16K,-0.169,-4,0.791391,43.708551,2190.5,2408.0,2212.0,704.5,1548.912600
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,1,deletion,16,L,-,55.23,L16-,-0.250,-10,0.965555,53.327592,2056.5,2408.0,2080.0,39.0,578.029408
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,C,69.25,K17C,-1.277,-5,0.841131,58.248314,1032.0,2386.5,1725.5,313.0,916.913762
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,F,69.25,K17F,-1.353,-4,0.791391,54.803859,987.0,2386.5,2053.5,704.5,1183.913201


In [16]:
test_df.sort_values('rank_pow')

,seq_id,protein_sequence,pH,data_source,n_edits,edit_type,edit_idx,old_aa,new_aa,b_factor,mut_string,ddG,sub_score,score_adj,b_factor_adj,ddG_rank,b_factor_rank,b_factor_adj_rank,sub_score_rank,rank_pow
543,31933,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,123,L,D,98.50,L123D,-6.263,-6,0.880797,86.758512,1.0,137.0,86.0,122.5,25.602687
537,31927,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,122,I,D,98.62,I122D,-5.841,-6,0.880797,86.864208,12.0,18.5,81.5,122.5,30.072049
540,31930,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,122,I,G,98.62,I122G,-4.905,-6,0.880797,86.864208,71.0,18.5,81.5,122.5,54.390375
1760,33150,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,122,I,P,98.62,I122P,-5.705,-4,0.791391,78.047027,17.0,18.5,456.5,704.5,60.510938
1751,33141,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,123,L,P,98.50,L123P,-6.126,-4,0.791391,77.952060,3.0,137.0,481.5,704.5,66.156767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,32908,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,159,K,Q,88.42,K159Q,-0.194,0,0.417430,36.909142,2156.0,2335.5,2355.0,2265.0,2250.948773
790,32180,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,157,Q,K,88.94,Q157K,-0.102,0,0.417430,37.126206,2256.0,2308.0,2353.0,2265.0,2276.220701
778,32168,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,156,Q,E,88.01,Q156E,-0.133,0,0.417430,36.737996,2236.5,2348.0,2357.5,2265.0,2282.680176
2392,33782,VPVNPEPDATSVENVALRTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,1,substitution,17,K,R,69.25,K17R,-0.142,0,0.339244,23.492621,2226.0,2386.5,2412.0,2265.0,2291.491594


In [17]:
assert not test_df[submit_col].isna().any()
submit_df = pd.DataFrame({
    'seq_id': test_df.seq_id.values,
    'tm': test_df[submit_col].values,
})
submit_df.tm = submit_df.tm.fillna(0)
submit_df.to_csv('deepddg-ddg.csv', index=False)
submit_df.head()

,seq_id,tm
0,31390,1168.255811
1,31391,1548.912600
2,31392,578.029408
3,31393,916.913762
4,31394,1183.913201
